In [1]:
!pip install transformers[torch] optuna sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.1 MB/s eta 0:00:00


In [2]:
# load packages
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AdamW, get_linear_schedule_with_warmup
import time
import datetime
import random
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import re
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler
import math

torch.cuda.amp.autocast(enabled=True)

In [3]:
## Input data paths

## base data
base_path = "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/"
train_base_path = base_path + "df_train.csv"
val_base_path = base_path + "df_val.csv"
test_base_path = base_path + "df_test.csv"


## explanations
exp_path = "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5-for-explanation/"
train_explanations_path =  exp_path + "df_train_pred_exp.csv"
val_explanations_path =  exp_path + "df_val_pred_exp.csv"
test_explanations_path =  exp_path + "df_test_pred_exp.csv"


## Read base data
df_train_base = pd.read_csv(train_base_path).astype(str)[['prefix','input_text','target_text']]
df_val_base = pd.read_csv(val_base_path).astype(str)[['prefix','input_text','target_text']]
df_test_base = pd.read_csv(test_base_path).astype(str)[['prefix','input_text','target_text']]
df_train_base = df_train_base[df_train_base['prefix']=='label'].copy()
df_val_base = df_val_base[df_val_base['prefix']=='label'].copy()
df_test_base = df_test_base[df_test_base['prefix']=='label'].copy()
df_train_base.rename(columns={'input_text':'sentence'}, inplace=True)
df_val_base.rename(columns={'input_text':'sentence'}, inplace=True)
df_test_base.rename(columns={'input_text':'sentence'}, inplace=True)


## Read explanations data
df_train_exp = pd.read_csv(train_explanations_path)
df_val_exp = pd.read_csv(val_explanations_path)
df_test_exp = pd.read_csv(test_explanations_path)
df_train_exp.rename(columns={'input_text':'sentence', 'predicted':'predicted_exp'}, inplace=True)
df_val_exp.rename(columns={'input_text':'sentence', 'predicted':'predicted_exp'}, inplace=True)
df_test_exp.rename(columns={'input_text':'sentence', 'predicted':'predicted_exp'}, inplace=True)


## Check shapes
print("Base data")
print("Train: ", df_train_base.shape)
print("Val: ", df_val_base.shape)
print("Test: ", df_test_base.shape)


print("Explanations: ")
print("Train: ", df_train_exp.shape)
print("Val: ", df_val_exp.shape)
print("Test: ", df_test_exp.shape)

Base data
Train:  (14072, 3)
Val:  (1787, 3)
Test:  (1761, 3)
Explanations: 
Train:  (14057, 5)
Val:  (1786, 5)
Test:  (1759, 5)


In [4]:
## Output Paths
model_op_path = "/content/drive/MyDrive/CS4NLP-HateXplain/data/t5_modeling/t5_ip_sent_exp_v2/"


In [5]:
## Merge data function to get modeling data
def get_model_data(df_label, df_exp):
  df = df_label.merge(df_exp[['sentence','predicted_exp']], on = ['sentence'], how = 'inner')
  df = df.astype(str)
  df['input_text'] = df.apply(lambda row: row['sentence'] + '. ' + row['predicted_exp'], axis=1)
  df = df[['prefix','input_text','target_text']].copy()
  #df.drop_duplicates(subset=['input_text'], keep='first', inplace=True)
  return df

train_df = get_model_data(df_train_base, df_train_exp)
eval_df = get_model_data(df_val_base, df_val_exp)
test_df = get_model_data(df_test_base, df_test_exp)

print("Train shape: ", train_df.shape)
print("Val shape: ", eval_df.shape)
print("Test shape: ", test_df.shape)

Train shape:  (14071, 3)
Val shape:  (1786, 3)
Test shape:  (1759, 3)


In [7]:
train_df.isnull().sum()

prefix         0
input_text     0
target_text    0
dtype: int64

In [6]:
## Instantiate Tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [8]:
# tokenize the main text
def tokenize_corpus(df, tokenizer, max_len=300):
    # token ID storage
    input_ids = []
    # attension mask storage
    attention_masks = []
    # max len -- 512 is max
    max_len = max_len
    # for every document:
    for doc in df:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            doc,  # document to encode.
                            add_special_tokens=True,  # add tokens relative to model
                            max_length=max_len,  # set max length
                            truncation=True,  # truncate longer messages
                            pad_to_max_length=True,  # add padding
                            return_attention_mask=True,  # create attn. masks
                            return_tensors='pt'  # return pytorch tensors
                       )

        # add the tokenized sentence to the list
        input_ids.append(encoded_dict['input_ids'])

        # and its attention mask (differentiates padding from non-padding)
        attention_masks.append(encoded_dict['attention_mask'])

    return torch.cat(input_ids, dim=0), torch.cat(attention_masks, dim=0)


# create tokenized data - input_text
train_body_input_ids, train_body_attention_masks = tokenize_corpus(train_df['input_text'].values, tokenizer)
eval_body_input_ids, eval_body_attention_masks = tokenize_corpus(eval_df['input_text'].values, tokenizer)
test_body_input_ids, test_body_attention_masks = tokenize_corpus(test_df['input_text'].values, tokenizer)

# create tokenized data - target_text - max_len=2
train_target_input_ids, train_target_attention_masks = tokenize_corpus(train_df['target_text'].values, tokenizer, max_len=2)
eval_target_input_ids, eval_target_attention_masks = tokenize_corpus(eval_df['target_text'].values, tokenizer, max_len=2)
test_target_input_ids, test_target_attention_masks = tokenize_corpus(test_df['target_text'].values, tokenizer, max_len=2)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
train_df.isnull().sum()

prefix         0
input_text     0
target_text    0
dtype: int64

In [15]:
## Function to prepare dataset
def prepare_dataset(body_tokens, body_masks, target_token, target_masks):
  tensor_df = TensorDataset(body_tokens, body_masks, target_token, target_masks)
  return tensor_df

# create tensor data sets
train_dataset = prepare_dataset(train_body_input_ids, train_body_attention_masks, train_target_input_ids, train_target_attention_masks)
eval_dataset = prepare_dataset(eval_body_input_ids, eval_body_attention_masks, eval_target_input_ids, eval_target_attention_masks)
test_dataset = prepare_dataset(test_body_input_ids, test_body_attention_masks, test_target_input_ids, test_target_attention_masks)


In [16]:
## Instantiate training models
## Training
def train(model, dataloader, optimizer):

    # capture time
    total_t0 = time.time()

    # Perform one full pass over the training set.
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
    print('Training...')

    # reset total loss for epoch
    train_total_loss = 0
    total_train_f1 = 0

    # put model into traning mode
    model.train()

    # for each batch of training data...
    for step, batch in enumerate(dataloader):

        # progress update every 40 batches.
        if step % 40 == 0 and not step == 0:

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(dataloader)))
            #print('Train loss: ', train_total_loss)

        # Unpack this training batch from our dataloader:
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input tokens
        #   [1]: attention masks
        #   [2]: target tokens
        #   [3]: target attenion masks
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_target_ids = batch[2].cuda()
        b_target_mask = batch[3].cuda()

        # clear previously calculated gradients
        optimizer.zero_grad()

        # runs the forward pass with autocasting.
        with autocast():
            # forward propagation (evaluate model on training batch)
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_target_ids,
                            decoder_attention_mask=b_target_mask)

            loss, prediction_scores = outputs[:2]

            # sum the training loss over all batches for average loss at end
            # loss is a tensor containing a single value
            # print("loss: ", loss)
            if math.isnan(loss.item())==False:
              train_total_loss += loss.item()

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.

        scaler.scale(loss).backward()

        # Clip gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5)

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(optimizer)

        # Updates the scale for next iteration.
        scaler.update()

        # update the learning rate
        scheduler.step()

    # calculate the average loss over all of the batches
    avg_train_loss = train_total_loss / len(dataloader)

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'Train Loss': avg_train_loss,
            'Total train loss': train_total_loss
        }
    )

    # training time end
    training_time = format_time(time.time() - total_t0)

    # print result summaries
    print("")
    print("summary results")
    print("epoch | trn loss | total trn loss | trn time ")
    print(f"{epoch+1:5d} | {avg_train_loss:.5f} | {train_total_loss:.5f} | {training_time:}")

    return training_stats


In [17]:
## Validation
def validating(model, dataloader):

    # capture validation time
    total_t0 = time.time()

    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")

    # put the model in evaluation mode
    model.eval()

    # track variables
    total_valid_loss = 0

    # evaluate data for one epoch
    for batch in dataloader:

        # Unpack this training batch from our dataloader:
        # `batch` contains three pytorch tensors:
        #   [0]: input tokens
        #   [1]: attention masks
        #   [2]: target tokens
        #   [3]: target attenion masks
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_target_ids = batch[2].cuda()
        b_target_mask = batch[3].cuda()

        # tell pytorch not to bother calculating gradients
        # as its only necessary for training
        with torch.no_grad():

            # forward propagation (evaluate model on training batch)
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_target_ids,
                            decoder_attention_mask=b_target_mask)

            loss, prediction_scores = outputs[:2]

            # sum the training loss over all batches for average loss at end
            # loss is a tensor containing a single value
            total_valid_loss += loss.item()

    # calculate the average loss over all of the batches.
    global avg_val_loss
    avg_val_loss = total_valid_loss / len(dataloader)

    # Record all statistics from this epoch.
    valid_stats.append(
        {
            'Val Loss': avg_val_loss,
            'Val PPL.': np.exp(avg_val_loss)
        }
    )

    # capture end validation time
    training_time = format_time(time.time() - total_t0)

    # print result summaries
    print("")
    print("summary results")
    print("epoch | val loss | val ppl | val time")
    print(f"{epoch+1:5d} | {avg_val_loss:.5f} | {np.exp(avg_val_loss):.3f} | {training_time:}")

    return valid_stats


In [18]:
## Testing
def testing(model, dataloader):

    print("")
    print("Running Testing...")

    # measure training time
    t0 = time.time()

    # put the model in evaluation mode
    model.eval()

    # track variables
    total_test_loss = 0
    total_test_acc = 0
    total_test_f1 = 0
    predictions = []
    actuals = []
    all_prediction_scores = []

    # evaluate data for one epoch
    for step, batch in enumerate(dataloader):
        # progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(dataloader), elapsed))

        # Unpack this training batch from our dataloader:
        # `batch` contains three pytorch tensors:
        #   [0]: input tokens
        #   [1]: attention masks
        #   [2]: target tokens
        #   [3]: target attenion masks
        b_input_ids = batch[0].cuda()
        b_input_mask = batch[1].cuda()
        b_target_ids = batch[2].cuda()
        b_target_mask = batch[3].cuda()

        # tell pytorch not to bother calculating gradients
        # as its only necessary for training
        with torch.no_grad():

            # forward propagation (evaluate model on training batch)
            outputs = model(input_ids=b_input_ids,
                            attention_mask=b_input_mask,
                            labels=b_target_ids,
                            decoder_attention_mask=b_target_mask)

            loss, prediction_scores = outputs[:2]

            total_test_loss += loss.item()

            generated_ids = model.generate(
                    input_ids=b_input_ids,
                    attention_mask=b_input_mask,
                    max_length=3
                    )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in b_target_ids]

            total_test_acc += accuracy_score(target, preds)
            total_test_f1 += f1_score(preds, target,
                                       average='weighted',
                                       labels=np.unique(preds))
            predictions.extend(preds)
            actuals.extend(target)
            all_prediction_scores.extend(prediction_scores)

    # calculate the average loss over all of the batches.
    avg_test_loss = total_test_loss / len(dataloader)

    avg_test_acc = total_test_acc / len(test_dataloader)

    avg_test_f1 = total_test_f1 / len(test_dataloader)

    # Record all statistics from this epoch.
    test_stats.append(
        {
            'Test Loss': avg_test_loss,
            'Test PPL.': np.exp(avg_test_loss),
            'Test Acc.': avg_test_acc,
            'Test F1': avg_test_f1
        }
    )
    global df2
    temp_data = pd.DataFrame({'predicted': predictions, 'actual': actuals})
    df2 = df2.append(temp_data)

    return test_stats, all_prediction_scores


In [19]:
# time function
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [20]:
## Prepare for training
model = T5ForConditionalGeneration.from_pretrained('t5-base').cuda()  # to GPU

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=False)

valid_dataloader = DataLoader(eval_dataset, batch_size=24, shuffle=False)

test_dataloader = DataLoader(test_dataset, batch_size=24, shuffle=False)


# Adam w/ Weight Decay Fix
# set to optimizer_grouped_parameters or model.parameters()
optimizer = AdamW(model.parameters(), lr = 1e-4)

# epochs
epochs = 5

# lr scheduler
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# create gradient scaler for mixed precision
scaler = GradScaler()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# create training result storage
training_stats = []
valid_stats = []
best_valid_loss = float('inf')

# for each epoch
for epoch in range(epochs):
    # train
    train(model, train_dataloader, optimizer)
    # validate
    validating(model, valid_dataloader)
    # check validation loss
    if valid_stats[epoch]['Val Loss'] < best_valid_loss:
        best_valid_loss = valid_stats[epoch]['Val Loss']
        # save best model for use later
        torch.save(model.state_dict(), model_op_path + 't5-classification.pt')  # torch save
        model_to_save = model.module if hasattr(model, 'module') else model
        model_to_save.save_pretrained(model_op_path + '/model_save/t5-classification/')  # transformers save
        tokenizer.save_pretrained(model_op_path + '/model_save/t5-classification/')  # transformers save


======== Epoch 1 / 5 ========
Training...
  Batch    40  of    880.
  Batch    80  of    880.
  Batch   120  of    880.
  Batch   160  of    880.
  Batch   200  of    880.
  Batch   240  of    880.
  Batch   280  of    880.
  Batch   320  of    880.
  Batch   360  of    880.
  Batch   400  of    880.
  Batch   440  of    880.
  Batch   480  of    880.
  Batch   520  of    880.
  Batch   560  of    880.
  Batch   600  of    880.
  Batch   640  of    880.
  Batch   680  of    880.
  Batch   720  of    880.
  Batch   760  of    880.
  Batch   800  of    880.
  Batch   840  of    880.

summary results
epoch | trn loss | total trn loss | trn time 
    1 | 1.37806 | 1212.69326 | 0:02:44

Running Validation...

summary results
epoch | val loss | val ppl | val time
    1 | 0.38688 | 1.472 | 0:00:11

======== Epoch 2 / 5 ========
Training...
  Batch    40  of    880.
  Batch    80  of    880.
  Batch   120  of    880.
  Batch   160  of    880.
  Batch   200  of    880.
  Batch   240  of    880

In [22]:
with torch.no_grad():
  torch.cuda.empty_cache()

In [23]:
# test the model
df2 = pd.DataFrame({'predicted': [], 'actual': []})
test_stats = []

# Load best model
model.load_state_dict(torch.load(model_op_path + 't5-classification.pt'))

<All keys matched successfully>

In [24]:
test_stats, all_prediction_scores = testing(model, test_dataloader)


Running Testing...
  Batch    40  of     74.    Elapsed: 0:00:16.


<ipython-input-18-1ad616789eb2>:90: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(temp_data)


In [25]:
test_stats, df2

([{'Test Loss': 0.3728285603023864,
   'Test PPL.': 1.4518354162709446,
   'Test Acc.': 0.683075933075933,
   'Test F1': 0.6873662468686046}],
       predicted     actual
 0        normal     normal
 1          hate       hate
 2        normal     normal
 3     offensive  offensive
 4          hate  offensive
 ...         ...        ...
 1754     normal     normal
 1755     normal  offensive
 1756     normal     normal
 1757     normal     normal
 1758  offensive  offensive
 
 [1759 rows x 2 columns])

In [26]:
correct = (df2['predicted'].apply(lambda x: x.strip()) == df2['actual'].apply(lambda x: x.strip())).sum()
acc = correct/df2.shape[0]
acc

0.6827743035815804